# Emotion Detection using TensorFlow and Transfer Learning
This notebook implements a real-time emotion detection system using TensorFlow, transfer learning (MobileNetV2), and OpenCV for webcam input.

In [ ]:
# Install required packages
!pip install tensorflow opencv-python matplotlib

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

## Data Preparation
Assuming the data is in a directory structure like:
`data/train/emotion_class/` and `data/validation/emotion_class/`

In [ ]:
# Create ImageDataGenerators
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_gen.flow_from_directory(
    'data/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

## Build Model with MobileNetV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Train the Model

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=10)

## Save the Model

In [ ]:
model.save('emotion_model_mobilenet')

## Real-Time Emotion Detection with OpenCV

In [ ]:
model = tf.keras.models.load_model('emotion_model_mobilenet')
class_names = list(train_data.class_indices.keys())

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi_color, (224, 224)) / 255.0
        roi_expanded = np.expand_dims(roi_resized, axis=0)
        prediction = model.predict(roi_expanded)
        emotion = class_names[np.argmax(prediction)]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()